# Upload API folder to IBM Cloud bucket

This is a temporary notebook for uploading all the files in the /api folder to a bucket on IBM Cloud

In [3]:
# Imports and general variables

import sys
import ibm_boto3
from ibm_botocore.client import Config
from pathlib import Path
import json
from tqdm import tqdm

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths


In [4]:
# Config

cred_file = Path(root_path / 'apikey.json')

with cred_file.open("r") as file:
    credentials = json.load(file)

In [5]:
# IBM COS configuration

cos = ibm_boto3.client(
    "s3",
    ibm_api_key_id=credentials['apikey'],
    ibm_service_instance_id="46e90cd7-db18-4c5a-8a1d-87d95250c725",
    config=Config(signature_version="oauth"),
    endpoint_url="https://s3.eu-de.cloud-object-storage.appdomain.cloud"
)

bucket_name = 'et-vgr-data'

In [7]:
directory = Path(paths.api_path)

all_files = [file_path for file_path in directory.rglob('*') if file_path.is_file()]


with tqdm(total=len(all_files), desc="Uploading files", unit="file") as pbar:
    for file_path in all_files:
        key = file_path.relative_to(directory).as_posix()

        with file_path.open("rb") as data:
            cos.upload_fileobj(data, bucket_name, key)
        
        pbar.update(1)
        pbar.set_postfix(file=key)

Uploading files: 100%|██████████| 36758/36758 [1:15:54<00:00,  8.07file/s, file=geography=1480,target-year=2030,self-sufficiency=1,energy-scenario=-0.2,h2=True,offwind=True,biogas-limit=0.5/performance/performance_metrics.csv.gz]                                                                                
